In [2]:
import logging
import telegram
import emoji
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from binance.client import Client
from binance.exceptions import BinanceAPIException
from secrets import secrets

TELEGRAM_CHAT_ID = secrets['TELEGRAM_CHAT_ID']
TELEGRAM_TOKEN = secrets['TELEGRAM_TOKEN']
BINANCE_API_KEY = secrets['BINANCE_API_KEY']
BINANCE_SECRET_KEY = secrets['BINANCE_SECRET_KEY']
BRIDGE = 'USDT'
COINS = ['BTC', 'ETH', 'LTC', 'XRP', 'BNB', 'ADA', 'BAT', 'FTM']

In [3]:
CHECK_TEXT = """{} :full_moon: - {} :dollar:
:clock1: Current Price:\t{}
:clock5: 5min Average Price:\t{}
:clock11: 24hr Average Price:\t{}
:repeat: 24hr Price Change:\t{}
:heavy_division_sign: 24hr Price Change %:\t{}
:unlock: Open Price:\t\t{}
:chart_with_upwards_trend: High Price:\t\t{}
:chart_with_downwards_trend: Low Price:\t\t{}
:lock: Close Price:\t\t{}
"""

UP_TEMPLATE = """:heavy_exclamation_mark: :heavy_exclamation_mark: :heavy_exclamation_mark: Heavy Swing Alert :heavy_exclamation_mark: :heavy_exclamation_mark: :heavy_exclamation_mark:
{} UP :arrow_upper_right: :arrow_upper_right: :arrow_upper_right:
BY {}% :ox:
:unlock: Open Price:\t\t{}
:lock: Close Price:\t\t{}
:chart_with_upwards_trend: Price Difference:\t{}
"""

DOWN_TEMPLATE = """:heavy_exclamation_mark: :heavy_exclamation_mark: :heavy_exclamation_mark: Heavy Swing Alert :heavy_exclamation_mark: :heavy_exclamation_mark: :heavy_exclamation_mark:
{} DOWN :arrow_lower_right: :arrow_lower_right: :arrow_lower_right:
BY {}% :bear:
:unlock: Open Price:\t\t{}
:lock: Close Price:\t\t{}
:chart_with_downwards_trend: Price Difference:\t{}
"""

In [4]:
def help(update, context):
    coins = ', '.join(COINS)
    update.message.reply_text('Coins available: {}\nUse \check to check status of coin.')

def check(update, context):
    try:
        coin = context.args[0].upper()
        if coin in COINS:
            get_coin_price(update, context, coin)
        else:
            update.message.reply_text(text='Please provide a valid coin.')
    except:
        update.message.reply_text(text='Error encountered. Please provide a valid coin.')

def period_price_check(context):
    client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
    for coin in COINS:
        text = get_price_change(coin, client)
        if len(text) > 0:
            context.bot.send_message(chat_id=TELEGRAM_CHAT_ID, text=emoji.emojize(text, use_aliases=True))

def get_coin_price(update, context, coin):
    client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
    symbol = coin + BRIDGE
    avg_price = client.get_avg_price(symbol=symbol)['price']
    t = client.get_ticker(symbol=symbol)
    text = TEXT_TEMPLATE.format(coin, BRIDGE, t['lastPrice'], avg_price, t['weightedAvgPrice'], t['priceChange'], 
                                t['priceChangePercent'], t['openPrice'], t['highPrice'], t['lowPrice'],
                                t['prevClosePrice'])
    update.message.reply_text(text=emoji.emojize(text, use_aliases=True))

def get_price_change(coin, client):
    symbol = coin + BRIDGE
    kline = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_15MINUTE, limit=1)[0]
    open_price = float(kline[1])
    close_price = float(kline[4])
    price_diff = close_price - open_price
    price_diff_percent = price_diff / open_price * 100
    if abs(price_diff_percent) >= 5:
        if price_diff_percent > 0:
            return UP_TEMPLATE.format(coin, price_diff_percent, open_price, close_price, price_diff)
        else:
            return DOWN_TEMPALTE.format(coin, abs(price_diff_percent), open_price, close_price, price_diff)
    else:
        return ''

def main():
    updater = Updater(token=TELEGRAM_TOKEN)
    # Bot commands
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('help', help))
    dp.add_handler(CommandHandler('check', check))
    # Job queue
    job_queue = updater.job_queue
    job_queue.run_repeating(period_price_check, interval=480, first=20) # Run every 8 minutes
    updater.start_polling()
    updater.idle()

In [5]:
main()

NameError: name 'btc' is not defined